In [ ]:
import os

# Asegurarse que el directorio de trabajo es la raíz del proyecto
import os
os.chdir(r'D:\ITMeet\Operaciones\BP010-data-pipelines-auditoria')
print(f'Working directory: {os.getcwd()}')
    
print(f"📍 Current working directory: {os.getcwd()}")

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine, text, pool, insert
from dotenv import load_dotenv
from datetime import datetime, timedelta

import logging

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

load_dotenv()

In [ ]:
def execute_sql(engine, esquema_stage_sql):
    """
    Ejecuta el DDL de Stage para crear todas las tablas.
    """
    try:
        with engine.connect() as connection:

            connection.execute(text(esquema_stage_sql))
            
            connection.commit()
        logger.info("consulta ejecutada exitosamente.")
    except Exception as e:
        logger.error(f"Error al crear tablas de Stage: {e}")
        raise

In [ ]:
# Es buena práctica usar variables de entorno para las credenciales
DB_USER = os.getenv("DB_USER", "audit")
DB_PASSWORD = os.getenv("DEV_DB_PASSWORD", "audit")
DB_NAME = os.getenv("DB_NAME", "etl_data")

# --- PARÁMETROS DEL TÚNEL SSH ---
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")

# 1. Construir la cadena de conexión
STAGE_DATABASE_URL = (
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@"
    f"{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

print(f"Stage Database URL: {STAGE_DATABASE_URL}")

In [ ]:
# Crear el motor y ejecutar
stage_engine = create_engine(STAGE_DATABASE_URL)

In [ ]:
with stage_engine.connect() as connection:
    response = connection.execute(text("""SELECT * FROM stage.tbl_pozo_produccion"""))
    tbl_pozo_produccion = response.fetchall()

In [ ]:
tbl_pozo_produccion

In [ ]:
with stage_engine.connect() as connection:
    response = connection.execute(text("""SELECT
    l.unit_id, 
    m.well_id,
    COUNT(*) AS num_registros
FROM 
    stage.landing_scada_data l
JOIN 
    stage.tbl_pozo_maestra m ON l.unit_id = m.well_id
GROUP BY
    l.unit_id, m.well_id
LIMIT 10;"""))

In [ ]:
rows = response.fetchall()
rows

In [ ]:
with open("src/sql/process/V1__stage_to_stage.sql", "r") as file:
    stage_to_bl_pozo_operacion_scada_sql = file.read()

In [ ]:
with stage_engine.connect() as connection:
    result = connection.execute(text(stage_to_bl_pozo_operacion_scada_sql))
    connection.commit()
    logger.info("Transformación de Stage a Stage ejecutada exitosamente.")

In [ ]:
with stage_engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM stage.tbl_pozo_produccion;"))
    logger.info("Transformación de Stage a Stage ejecutada exitosamente.")

In [ ]:
result.fetchall()